In [1]:
from utils.db_connector import DBConnector
from typing import List

In [2]:
dbconnector = DBConnector()

In [3]:
dbconnector.connect()

Connection to the database established successfully.


<connection object at 0x000001BDBF954D00; dsn: 'user=postgres password=xxx dbname=insurancedb host=4.213.75.90 port=5432', closed: 0>

In [ ]:
select_query = "SELECT * from policies;"
results = dbconnector.execute_query(select_query, fetch_results=True)

In [5]:
from utils.models import Customer,Nominee,Policy,Endorsement,Claim,ConversationHistory,convert_db_row_to_model

In [6]:
policy_columns = [
        'policy_id', 'customer_id', 'policy_type', 'insurer_name',
        'start_date', 'end_date', 'sum_insured', 'premium_amount', 'policy_status'
    ]
customers = []
for row in results:
    customer = convert_db_row_to_model(row, Policy)
    customers.append(customer)

In [7]:
for rows in results:
    print(rows)

('POL2023001', 1, 'Health', 'Star Health Insurance', datetime.date(2023, 1, 15), datetime.date(2024, 1, 14), Decimal('500000.00'), Decimal('15000.00'), 'active', datetime.datetime(2025, 7, 2, 11, 26, 4, 741164, tzinfo=datetime.timezone.utc))
('POL2023002', 2, 'Car', 'ICICI Lombard', datetime.date(2023, 3, 20), datetime.date(2024, 3, 19), Decimal('750000.00'), Decimal('22000.00'), 'active', datetime.datetime(2025, 7, 2, 11, 26, 4, 741164, tzinfo=datetime.timezone.utc))
('POL2022003', 3, 'Life', 'LIC', datetime.date(2022, 6, 1), datetime.date(2042, 6, 1), Decimal('10000000.00'), Decimal('50000.00'), 'active', datetime.datetime(2025, 7, 2, 11, 26, 4, 741164, tzinfo=datetime.timezone.utc))
('POL2023004', 4, 'Fire', 'HDFC ERGO', datetime.date(2023, 5, 10), datetime.date(2024, 5, 9), Decimal('2500000.00'), Decimal('8000.00'), 'expired', datetime.datetime(2025, 7, 2, 11, 26, 4, 741164, tzinfo=datetime.timezone.utc))
('POL2023005', 1, 'Car', 'Go Digit', datetime.date(2023, 8, 1), datetime.date

In [8]:
customers

[Policy(policy_id='POL2023001', customer_id=1, policy_type='Health', insurer_name='Star Health Insurance', start_date='2023-01-15', end_date='2024-01-14', sum_insured=500000.0, premium_amount=15000.0, policy_status='active'),
 Policy(policy_id='POL2023002', customer_id=2, policy_type='Car', insurer_name='ICICI Lombard', start_date='2023-03-20', end_date='2024-03-19', sum_insured=750000.0, premium_amount=22000.0, policy_status='active'),
 Policy(policy_id='POL2022003', customer_id=3, policy_type='Life', insurer_name='LIC', start_date='2022-06-01', end_date='2042-06-01', sum_insured=10000000.0, premium_amount=50000.0, policy_status='active'),
 Policy(policy_id='POL2023004', customer_id=4, policy_type='Fire', insurer_name='HDFC ERGO', start_date='2023-05-10', end_date='2024-05-09', sum_insured=2500000.0, premium_amount=8000.0, policy_status='expired'),
 Policy(policy_id='POL2023005', customer_id=1, policy_type='Car', insurer_name='Go Digit', start_date='2023-08-01', end_date='2024-07-31',